<hr style="border:2px solid gray"> </hr>
<img src="https://mma.prnewswire.com/media/1095203/East_Tennessee_State_University_Logo.jpg?p=facebook" width=200 height=200 />

<div class="alert alert-block alert-info">
    <h1 style="text-align: center">CSCI 5037 - NLP & Text Analysis</h1>
</div>

# <center>Lab 3 – Named Entity Recognition</center>

**<center>Dr. Ahmad Al-Doulat </center>**
<center>Department of Computing </center>
<center>East Tennessee State University</center>

<hr style="border:2px solid lightblue"> </hr>

**In this assignment, you'll get to practice the concepts and skills covered in the first 3 modules (Modules 1, 2, and 3). The main objective of this assignment is to implement and use some of the tools, algorithms, and techniques to represent and clean textual data and to extract named entities..**



**Guidelines**
* Download `NER.csv` file from D2L.
* Make sure to run all the code cells, otherwise you may get errors like `NameError` for undefined variables.
* Do not change variable names, delete cells or disturb other existing code. It may cause problems during evaluation.
* In some cases, you may need to add some code cells or new statements before or after the line of code containing the `???`.
* Use markdown cells to write your discussions and reflections.

**Procedure**
* Save your work as `IPYNB` file named `Lab3.ipynb` and submit to D2L `Lab 3 – Named Entity Recognition (Dropbox)` by the due date.
* As you go through this notebook, you will find the symbol `???` in certain places. To complete this assignment, you must replace all the `???` with appropriate values, expressions or statements to ensure that the notebook runs properly end-to-end.
* Include your response for `Part 1` and `Part 2` in this notebook.

<div class="alert alert-block alert-info">

# Part 1: Activity

</div>

# Question 1: Reading the dataset
<hr style="border:1px solid orange"> </hr>

#### Read the content of the `NER.csv` into a dataframe `ner_df` and perform the following:

> **Q1.1.** Perform Part-of-Speech tagging on the `Sentence` column in the `ner_df`. Then, add the results as new column called `pos`.

> **Q1.2.** Implement a function called `np_chunker` that receives each sentence from the `ner_df` dataframe. Then, it defines and applies a chunk parser to chunk all noun phrases from these sentences.

> **Q1.3.** Perfrom a Named Entity Recognition (NER) on the Sentence column and add the extracted entities as a new column called `entities`

In [24]:
from IPython.display import display

from collections import Counter
from pprint import pprint

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import tqdm as tq
from tqdm import tqdm

import string

import nltk

from nltk.corpus import treebank
from nltk.tag import hmm

!pip install watermark
import watermark

%load_ext watermark
%matplotlib inline

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [25]:
# Downloading the necessary nltk packages for our code.

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('treebank')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

#### Q1.1. Perform Part-of-Speech tagging on the Sentence column in the ner_df. Then, add the results as new column called pos.

In [26]:
# Reading the data from ner.csv file using pandas

ner = pd.read_csv('sample_data/ner.csv')
ner_df = pd.DataFrame(ner)
ner_df

# Building a function that takes sentence as argument and returns parts of speech tags.

def pos_tag(sentence):
  tokens = nltk.word_tokenize(sentence)
  pos_tags = nltk.pos_tag(tokens)
  return pos_tags

# Adding the result into a new "pos" column to the ner.csv file.

ner_df['pos'] = ner_df['Sentence'].apply(pos_tag)
ner_df

,Sentence #,Sentence,pos
0,Sentence: 1,Thousands of demonstrators have marched throug...,"[(Thousands, NNS), (of, IN), (demonstrators, N..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"[(Families, NNS), (of, IN), (soldiers, NNS), (..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"[(They, PRP), (marched, VBD), (from, IN), (the..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","[(Police, NNP), (put, VBD), (the, DT), (number..."
4,Sentence: 5,The protest comes on the eve of the annual con...,"[(The, DT), (protest, NN), (comes, VBZ), (on, ..."
...,...,...,...
47954,Sentence: 47955,Indian border security forces are accusing the...,"[(Indian, JJ), (border, NN), (security, NN), (..."
47955,Sentence: 47956,Indian officials said no one was injured in Sa...,"[(Indian, JJ), (officials, NNS), (said, VBD), ..."
47956,Sentence: 47957,Two more landed in fields belonging to a nearb...,"[(Two, CD), (more, JJR), (landed, VBN), (in, I..."
47957,Sentence: 47958,They say not all of the rockets exploded upon ...,"[(They, PRP), (say, VBP), (not, RB), (all, DT)..."


#### Q1.2. Implement a function called np_chunker that receives each sentence from the ner_df dataframe. Then, it defines and applies a chunk parser to chunk all noun phrases from these sentences.

In [27]:
# Building a np_chunker function that takes each sentence as argument,
# chunks all the noun phrases from the sentences.

def np_chunker(sentence):
  tokens = nltk.word_tokenize(sentence)
  pos_tags = nltk.pos_tag(tokens)
  grammar = r"""NP: {<DT>?<JJ.*>*<NN.*>+}"""
  chunk_parser = nltk.RegexpParser(grammar)
  tree = chunk_parser.parse(pos_tags)
  return tree

# Printing the result
  for subtree in tree.subtrees():
    print(subtree)

# Calling the np_chunker function with the Sentece column as argument
ner_df['Sentence'].apply(np_chunker)

,Sentence
0,"[[(Thousands, NNS)], (of, IN), [(demonstrators..."
1,"[[(Families, NNS)], (of, IN), [(soldiers, NNS)..."
2,"[(They, PRP), (marched, VBD), (from, IN), [(th..."
3,"[[(Police, NNP)], (put, VBD), [(the, DT), (num..."
4,"[[(The, DT), (protest, NN)], (comes, VBZ), (on..."
...,...
47954,"[[(Indian, JJ), (border, NN), (security, NN), ..."
47955,"[[(Indian, JJ), (officials, NNS)], (said, VBD)..."
47956,"[(Two, CD), (more, JJR), (landed, VBN), (in, I..."
47957,"[(They, PRP), (say, VBP), (not, RB), (all, DT)..."


#### Q1.3. Perform a Named Entity Recognition (NER) on the Sentence column and add the extracted entities as a new column called entities

In [28]:
# Building a ner_tag function that takes sentence as argument
# and returns entities with the corresponding (NER) labels.

def ner_tag(sentence):
  tokens = nltk.word_tokenize(sentence)
  pos_tags = nltk.pos_tag(tokens)
  ner_tags = nltk.ne_chunk(pos_tags)

  for chunk in ner_tags:
    if hasattr(chunk, 'label'):
      return chunk.label(), ' '.join(c[0] for c in chunk)

# Adding the result into a new column called "entities" in the ner_df dataframe

ner_df['entities'] = ner_df['Sentence'].apply(ner_tag)
ner_df['entities']

,entities
0,"(GPE, London)"
1,"(PERSON, Bush Number One Terrorist)"
2,"(ORGANIZATION, Houses)"
3,None
4,"(GPE, Britain)"
...,...
47954,"(GPE, Indian)"
47955,"(GPE, Indian)"
47956,None
47957,None


<div class="alert alert-block alert-info">

# Part 2: Reflection
    
</div>

As a second step—after answering the questions, include the following:
1. A reflection of your experience performing the activity.
2. A reflection on the importance of learning this activity.


**Note:** include your reflection in this notebook as markdown cells.

**1. A reflection of your experience performing the activity.**

I found the Named entity relationship activity as both challenging and rewarding. As I started working on Named Entity Recognition (NER), it became clear that extracting entities and labeling them accurately requires a detailed understanding of the text and how machine learning models handle language processing. I got firsthand experience in breaking down complex information into structured data using NER, which was quite satisfying. I learned a lot about the nuances of preprocessing text, building models, and interpreting the labeled results.

**2. Reflection on the Importance of Learning the Activity**

NER and other NLP tasks are crucial in today’s data-driven world, especially when dealing with large volumes of unstructured data like text. It equips us with the ability to extract meaningful insights, identify key information, and even automate complex tasks. This knowledge is essential not only for advancing research but also for practical applications in industries where language understanding is key, such as customer service, content analysis, and even personal assistants.

<div class="alert alert-block alert-info">

# Submission
    
</div>

Submit **Lab3.ipynb** to the **Lab 3 – Named Entity Recognition (Dropbox)** on D2L by the due date.

<div class="alert alert-block alert-info">

# Grading Rubric
    
</div>

| Criterion                                | Excellent                                          | Good                                               | Average                                           | Below Average                                   | Poor                                              | No Attempt                      |
|:---------------------------------------- |:-------------------------------------------------- |:-------------------------------------------------- |:------------------------------------------------ |:------------------------------------------------ |:------------------------------------------------ |:-------------------------------- |
| **Part 1:** Activity-Question 1.1        | **10 points** - Completes all aspects of the question correctly | **8 points** - Completes most aspects of the question correctly | **6 points** - Completes aspects of the question correctly, and some incorrectly | **4 points** - Completes most aspects of the question incorrectly or does not attempt many aspects | **2 points** - Minimal effort or completes a few aspects of the question or very few correctly | **0 points** - Did not complete the question |
| **Part 1:** Activity-Question 1.2        | **10 points** - Completes all aspects of the question correctly | **8 points** - Completes most aspects of the question correctly | **6 points** - Completes aspects of the question correctly, and some incorrectly | **4 points** - Completes most aspects of the question incorrectly or does not attempt many aspects | **2 points** - Minimal effort or completes a few aspects of the question or very few correctly | **0 points** - Did not complete the question |
| **Part 1:** Activity-Question 1.3        | **10 points** - Completes all aspects of the question correctly | **8 points** - Completes most aspects of the question correctly | **6 points** - Completes aspects of the question correctly, and some incorrectly | **4 points** - Completes most aspects of the question incorrectly or does not attempt many aspects | **2 points** - Minimal effort or completes a few aspects of the question or very few correctly | **0 points** - Did not complete the question |
| **Part 2:** Reflection                   | **10 points** - Reflection clearly ties to the module content; experience and importance clearly laid out | **8 points** - Reflection mostly ties to the module content; experience & importance are discussed | **6 points** - Reflection ties minimally to the module content; experience & importance are discussed but not thoroughly | **4 points** - Reflection does not tie to the module content; experience & importance are minimally discussed | **2 points** - Minimal effort to tie to content; minimal effort to describe experience/importance | **0 points** - Did not complete the reflection |
